In [1]:
import logging
import tensorflow as tf


GPU_FROM = 0
GPU_TO = 1    

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs visible:{len(visible_devices)}")
tf.config.set_visible_devices(visible_devices[GPU_FROM:GPU_TO],'GPU')

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs to be used: {len(visible_devices)}")

from segmentation.model import SpaceSegmentationTransformer
from segmentation.model import LossWithVoids

data = tf.random.stateless_binomial(shape=(10000, 100), counts=1, probs=0.8, seed=[1997,1997])
train_frac = int(data.shape[0]*3/4)

train_ds, val_ds = tf.data.Dataset.from_tensor_slices(data[:train_frac]), tf.data.Dataset.from_tensor_slices(data[train_frac:])

def mapper(y):
    x = tf.strings.as_string(y+4)
    x = tf.strings.reduce_join(x, axis=-1)
    return tf.strings.regex_replace(x, "(45)", r"\1 ")

train_ds = train_ds.map(mapper)
val_ds = val_ds.map(mapper)

for f in train_ds.take(5):
    print(f)

2023-02-21 17:01:41.026436: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 17:01:41.195371: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-21 17:01:41.234168: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-21 17:01:41.983549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

tf.Tensor(b'5555545 5555555445 5555445 545 5555545 555545 55555445 545 545 545 555545 5545 55555445 5555445 555545 55555555545 4', shape=(), dtype=string)
tf.Tensor(b'55545 5545 5545 45 555445 5445 45 5555555555545 545 555545 545 45 55545 55555545 55555545 545 45 45 445 45 55545 55555555', shape=(), dtype=string)
tf.Tensor(b'5555544445 45 55555545 45 5555445 555555545 555555555555555555545 555445 5555555545 5555545 5545 545 5445 45 45 444', shape=(), dtype=string)
tf.Tensor(b'545 555555445 555555545 555545 555555545 5555545 5555555555555445 45 45 555545 555545 5555555555555445 555555544', shape=(), dtype=string)
tf.Tensor(b'545 5555545 45 555555555445 554445 555545 555555555545 55555545 5545 555545 5555555445 55445 5555545 4445 5445 445 5', shape=(), dtype=string)


2023-02-21 17:01:43.002261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 17:01:43.664002: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22296 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


In [2]:
def generate_labels(text):
    max_chars = 100

    last_char = tf.strings.substr(text, max_chars, max_chars+1)
    text = tf.strings.substr(text,0, max_chars)
    text = tf.strings.lower(text)
    split_on_whitespace = tf.strings.strip(tf.strings.split(text))
    encoder_in = tf.strings.reduce_join(split_on_whitespace, axis=-1)
    
    space_indices = tf.math.cumsum(tf.strings.length(split_on_whitespace)) - 1 # subtract 1 to index from 0
    seq_len = space_indices[-1] + 1
    if not tf.strings.regex_full_match(last_char, " "):
        space_indices = space_indices[:-1]
    encoder_out = tf.ones((max_chars,))
    encoder_out = tf.concat([
        tf.ones((seq_len,)),
        tf.zeros((max_chars - seq_len,))
    ], axis=-1)
    encoder_out = tf.tensor_scatter_nd_update(
        encoder_out, 
        space_indices[...,tf.newaxis], # Expand dims to create a 'list' of indices
        tf.ones_like(space_indices, dtype=encoder_out.dtype)*2) 
    
    return (encoder_in, None), tf.cast(encoder_out-1, "float16")

train_ds = train_ds.map(generate_labels)
val_ds = val_ds.map(generate_labels)

for f in train_ds.take(1):
    print(f)

((<tf.Tensor: shape=(), dtype=string, numpy=b'55555455555555445555544554555555455555455555544554554554555554555455555544555554455555'>, None), <tf.Tensor: shape=(100,), dtype=float16, numpy=
array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.], dtype=float16)>)


In [3]:
tokenizer = tf.keras.layers.TextVectorization(
    output_sequence_length=100,
    standardize="lower_and_strip_punctuation",
    split="character",
    output_mode="int",
)

tokenizer.adapt(train_ds.take(10).map(lambda x,y: x[0]))
tokenizer.get_vocabulary()

['', '[UNK]', '5', '4']

In [4]:
tokenizer("44444455555")

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>

In [5]:
model = SpaceSegmentationTransformer(
    num_layers=2,
    d_model=512,
    num_attention_heads=3,
    seq_len=100,
    dff=1028,
    input_tokenizer=tokenizer,
    dropout_rate=0.1,
    num_classes = 2
)

In [6]:
from segmentation.metrics import SparseAccuracyWithIgnore
from segmentation.metrics import SparsePrecision
from segmentation.metrics import SparseRecall
from segmentation.metrics import SparseF1

model.compile(
    optimizer="adam",
    # TODO: Look into this loss function - specifically what ignore_class is doing.
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, ignore_class=-1), # Why can I not ignore class 0?
    metrics=[
        SparseAccuracyWithIgnore(ignore_token=-1),
        SparsePrecision(class_id=1, name="space_precision"),
        SparseRecall(class_id=1, name="space_recall"),
        SparseF1(class_id=1, name="space_f1"),
        SparsePrecision(class_id=0, name="char_precision"),
        SparseRecall(class_id=0, name="char_recall"),
        SparseF1(class_id=0, name="char_f1"),
    ],
)

In [7]:
train_ds = train_ds.shuffle(100).batch(8)
val_ds = val_ds.batch(8)

history = model.fit(train_ds, validation_data=val_ds, epochs=10)
model.summary()

Epoch 1/10
  7/938 [..............................] - ETA: 20s - loss: 3.4105 - sparse_categorical_accuracy: 0.6741 - space_precision: 0.1314 - space_recall: 0.2216 - space_f1: 0.1650 - char_precision: 0.7162 - char_recall: 0.7610 - char_f1: 0.7379   

2023-02-21 17:01:50.175498: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


938/938 [==============================] - 30s 27ms/step - loss: 0.4597 - sparse_categorical_accuracy: 0.8428 - space_precision: 0.1216 - space_recall: 0.0029 - space_f1: 0.0056 - char_precision: 0.7306 - char_recall: 0.9968 - char_f1: 0.8432 - val_loss: 0.4354 - val_sparse_categorical_accuracy: 0.8454 - val_space_precision: 0.0000e+00 - val_space_recall: 0.0000e+00 - val_space_f1: nan - val_char_precision: 0.7312 - val_char_recall: 1.0000 - val_char_f1: 0.8447
Epoch 2/10
938/938 [==============================] - 25s 26ms/step - loss: 0.4285 - sparse_categorical_accuracy: 0.8450 - space_precision: 0.1429 - space_recall: 1.9900e-05 - space_f1: 3.9794e-05 - char_precision: 0.7306 - char_recall: 1.0000 - char_f1: 0.8444 - val_loss: 0.3939 - val_sparse_categorical_accuracy: 0.8454 - val_space_precision: 0.0000e+00 - val_space_recall: 0.0000e+00 - val_space_f1: nan - val_char_precision: 0.7312 - val_char_recall: 1.0000 - val_char_f1: 0.8447
Epoch 3/10
938/938 [=============================

In [8]:
model.evaluate(val_ds)

313/313 [==============================] - 4s 13ms/step - loss: 0.4309 - sparse_categorical_accuracy: 0.8454 - space_precision: 0.0000e+00 - space_recall: 0.0000e+00 - space_f1: nan - char_precision: 0.7312 - char_recall: 1.0000 - char_f1: 0.8447


[0.4308864176273346,
 0.8454476594924927,
 0.0,
 0.0,
 nan,
 0.7312039732933044,
 1.0,
 0.8447346091270447]

In [9]:
preds = model([("5554445", "55544455554445", "5454545454545454"),(None,None, None)])
tf.argmax(preds, axis=-1)

<tf.Tensor: shape=(3, 100), dtype=int64, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [17]:
import numpy as np
embeddings = model.encoder.pos_embedding.embedding(tokenizer(['4', '5']))[:, 0, :].numpy()
norm = (embeddings**2).sum(axis=-1)**0.5
norm = norm[..., np.newaxis] * norm[np.newaxis,...]
np.arccos(embeddings@embeddings.T/norm)/np.pi*180

array([[1.9782340e-02, 8.0487526e+01],
       [8.0487526e+01, 4.4234656e-02]], dtype=float32)